## Reading Data from the STAC API
reference: https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/

There are rates limit

In [ ]:
#%pip install pystac-client
#%pip install planetary-computer
#%pip install geopandas
#%pip install "rich[jupyter]"
#%pip install rasterio

In [ ]:
import pystac_client
import planetary_computer


In [ ]:
#background step before execution:
# configure in terminal: planetarycomputer configure
#See: https://planetarycomputer.microsoft.com/docs/concepts/sas/

catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [ ]:
time_range = "2020-12-01/2020-12-31"
bbox = [-122.2751, 47.5469, -121.9613, 47.7458]

search = catalog.search(collections=["landsat-c2-l2"], bbox=bbox, datetime=time_range)
items = search.get_all_items()
len(items)

In [ ]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-122.2751, 47.5469],
            [-121.9613, 47.9613],
            [-121.9613, 47.9613],
            [-122.2751, 47.9613],
            [-122.2751, 47.5469],
        ]
    ],
}

time_range = "2020-12-01/2020-12-31"

search = catalog.search(
    collections=["landsat-c2-l2"], intersects=area_of_interest, datetime=time_range
)

In [ ]:
items

In [ ]:
import geopandas

df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df

In [ ]:
selected_item = min(items, key=lambda item: item.properties["eo:cloud_cover"])
selected_item

In [ ]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title)

table

In [ ]:
selected_item.assets["rendered_preview"].to_dict()

In [ ]:
from IPython.display import Image

Image(url=selected_item.assets["rendered_preview"].href, width=500)

In [ ]:
selected_item.assets["blue"].href

In [ ]:
import requests

requests.head(selected_item.assets["blue"].href).status_code

In [ ]:
selected_item

In [ ]:
url="""https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2020/047/027/LC08_L2SP_047027_20201204_20210313_02_T1/LC08_L2SP_047027_20201204_20210313_02_T1_SR_B7.TIF"""

In [ ]:
r = requests.get(url,stream = True)

In [ ]:
r.status_code

In [ ]:
import shutil

In [ ]:
fname = "LC08_L2SP_047027_20201204_20210313_02_T1_SR_B7.TIF"
if r.status_code == 200:
    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    r.raw.decode_content = True
    
    # Open a local file with wb ( write binary ) permission.
    with open(fname,'wb') as f:
        shutil.copyfileobj(r.raw, f)